In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import defaultdict, deque
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [2]:
def get_data():
    input_text = au.read_txt_file_lines()
    ind_split = np.where([x == '' for x in input_text])[0][0]
    init = []
    for ii in input_text[:ind_split]:
        tmp1, tmp2 = ii.split(':')
        tmp2 = int(tmp2.lstrip())
        init.append((tmp1, tmp2))

    gates = []
    dict_in = defaultdict(list)
    dict_out = defaultdict(list)

    for ig, gg in enumerate(input_text[ind_split + 1:]):
        tmp_in, tmp_out = gg.split('->')
        tmp_out = tmp_out.lstrip()
        in1, t, in2 = tmp_in.split()
        gates.append((in1, in2, tmp_out, t))

        dict_in[in1].append(ig)
        dict_in[in2].append(ig)
        dict_out[tmp_out].append(ig)
    return gates, init, dict_in, dict_out

gates, init, dict_in, dict_out = get_data()
print(f'N gates: {len(gates)}')


N gates: 222


In [3]:
## there are no loops ... just one loop then (for now?)
def eval_machine(gates, init, dict_in):
    vals = {ii[0]: ii[1] for ii in init}
    wires = {ig: [None, None, None] for ig in range(len(gates))}
    queue = deque(init)

    while len(queue) > 0:
        iw = queue.popleft()
        ind_gates = dict_in[iw[0]]

        for ig in ind_gates:
            ind_w = np.where([tmp_g == iw[0] for tmp_g in gates[ig]])[0][0]
            wires[ig][ind_w] = iw[1]

            if wires[ig][0] is not None and wires[ig][1] is not None:
                assert wires[ig][2] is None, wires[ig]
                assert wires[ig][0] in [0, 1] and wires[ig][1] in [0, 1]
                if gates[ig][3] == 'AND':
                    wires[ig][2] = wires[ig][0] & wires[ig][1]
                elif gates[ig][3] == 'OR':
                    wires[ig][2] = wires[ig][0] | wires[ig][1]
                elif gates[ig][3] == 'XOR':
                    wires[ig][2] = wires[ig][0] ^ wires[ig][1]
                vals[gates[ig][2]] = wires[ig][2]
                queue.append((gates[ig][2], wires[ig][2]))
    return vals, wires, gates

def join_bits(vals_z, verbose=True):
    vals_key = list(vals_z.keys())
    tmp = [v[0] for v in vals_key]
    assert len(set(tmp)) == 1, tmp
    lead_str = tmp[0]
    final_z = ''
    ind = 0 
    str_z = lead_str + str(ind).zfill(2) 
    while str_z in vals_z.keys():
        final_z = str(vals_z[str_z]) + final_z
        ind += 1
        str_z = lead_str + str(ind).zfill(2) 

    if verbose:
        print(final_z, int(final_z, 2))
    return final_z, int(final_z, 2)

vals, wires, gates = eval_machine(gates, init, dict_in)
vals_z = {k: v for k, v in vals.items() if k[0] == 'z'}
_ = join_bits(vals_z)


1101000110101010100101111010011000011100100110 57632654722854


## part 2

In [60]:
## Look for anomalies, this was the winning search: 
## (z45 OR is fine, others are wrong)

gates, init, dict_in, dict_out = get_data()

gates_and = []
for g in gates:
    if 'z' in g[2] and g[3] != 'XOR':
    # if 'x' in g[0] and 'y' in g[1] and g[3] == 'AND':
    # if 'x06' in g[0]:
        gates_and.append(g)

gates_and = sorted(gates_and, key=lambda g: g[2])
gates_and

[('x15', 'y15', 'z15', 'AND'),
 ('rqt', 'rdt', 'z23', 'OR'),
 ('vbt', 'vqr', 'z39', 'AND'),
 ('jwh', 'tmh', 'z45', 'OR')]

In [ ]:
## Just one extra pair needs to be found. Just do brute force N^2 possibilities:

def get_data_part2(extra_swap=None):
    wrong = [('qbw', 'fqf', 'ckj', 'XOR'), 
            ('x15', 'y15', 'z15', 'AND'), 
            ('nsr', 'gsd', 'kdf', 'XOR'),   
            ('rqt', 'rdt', 'z23', 'OR'),  
            ('vbt', 'vqr', 'z39', 'AND'),
            ('vqr', 'vbt', 'rpp', 'XOR')]
    
    right = [('qbw', 'fqf', 'z15', 'XOR'), 
            ('x15', 'y15', 'ckj', 'AND'), 
            ('nsr', 'gsd', 'z23', 'XOR'),   
            ('rqt', 'rdt', 'kdf', 'OR'),  
            ('vbt', 'vqr', 'rpp', 'AND'),
            ('vqr', 'vbt', 'z39', 'XOR')]
    
    input_text = au.read_txt_file_lines()
    ind_split = np.where([x == '' for x in input_text])[0][0]
    init = []
    for ii in input_text[:ind_split]:
        tmp1, tmp2 = ii.split(':')
        tmp2 = int(tmp2.lstrip())
        init.append((tmp1, tmp2))

    gates = []
    dict_in = defaultdict(list)
    dict_out = defaultdict(list)

    for ig, gg in enumerate(input_text[ind_split + 1:]):
        tmp_in, tmp_out = gg.split('->')
        tmp_out = tmp_out.lstrip()
        in1, t, in2 = tmp_in.split()
        gate = (in1, in2, tmp_out, t)
        if gate in wrong:  # swap 3 wrong paris
            ind_wrong_gate = np.where([x == gate for x in wrong])[0][0]

            gate = right[ind_wrong_gate]
        gates.append(gate)

        dict_in[in1].append(ig)
        dict_in[in2].append(ig)
        dict_out[tmp_out].append(ig)

    if extra_swap is not None:  # swap an arbitrary pair (i1, i2)
        ig1, ig2 = extra_swap
        g1 = list(gates[ig1])
        g2 = list(gates[ig2])

        g1s, g2s = g1, g2
        g1s[2], g2s[2] = g2[2], g1[2]

        gates[ig1] = tuple(g1s)
        gates[ig2] = tuple(g2s)

    return gates, init, dict_in, dict_out

gates, init, dict_in, dict_out = get_data_part2()
print(f'N gates: {len(gates)}')

vals_x = {ii[0]: ii[1] for ii in init if ii[0][0] == 'x'}
vals_y = {ii[0]: ii[1] for ii in init if ii[0][0] == 'y'}

vals, wires, gates = eval_machine(gates, init, dict_in)
vals_z = {k: v for k, v in vals.items() if k[0] == 'z'}

x_bin, x_int = join_bits(vals_x)
y_bin, y_int = join_bits(vals_y)
z_bin, z_int = join_bits(vals_z)
print(z_int)
print(x_int + y_int)

print('0' + x_bin)
print('0' + y_bin)
print(z_bin)

N gates: 222
111000011010110000101100011101101111011001101 31016199577293
101111011010100100000110110101010100000011001 26066707720217
1100111110101010100110011010011000011100100110 57082907297574
57082907297574
57082907297510
0111000011010110000101100011101101111011001101
0101111011010100100000110110101010100000011001
1100111110101010100110011010011000011100100110


In [56]:
n = len(gates)

for i1 in range(n - 1):
    for i2 in range(i1 + 1, n):


        gates, init, dict_in, dict_out = get_data_part2((i1, i2))
        
        vals_x = {ii[0]: ii[1] for ii in init if ii[0][0] == 'x'}
        vals_y = {ii[0]: ii[1] for ii in init if ii[0][0] == 'y'}

        vals, wires, gates = eval_machine(gates, init, dict_in)
        vals_z = {k: v for k, v in vals.items() if k[0] == 'z'}

        x_bin, x_int = join_bits(vals_x, 0)
        y_bin, y_int = join_bits(vals_y, 0)
        z_bin, z_int = join_bits(vals_z, 0)
        
        if z_int == (x_int + y_int):
            assert False, (i1, i2)


AssertionError: (75, 151)

In [57]:

gates, init, dict_in, dict_out = get_data_part2()
print(gates[i1], i1)
print(gates[i2], i2)

('y06', 'x06', 'fdv', 'AND') 75
('x06', 'y06', 'dbp', 'XOR') 151


In [61]:
wrong_outputs = ['ckj', 'z15', 'kdf', 'z23', 'z39', 'rpp', 'fdv', 'dbp']
','.join(sorted(wrong_outputs))


'ckj,dbp,fdv,kdf,rpp,z15,z23,z39'

## some misc code:

In [54]:
print_queue = deque([])

def find_ind_g(w):
    tmp = []
    for ind_g in dict_in[w]:
        tmp.append(ind_g)
    
    for ind_g in dict_out[w]:
        tmp.append(ind_g)

    return tmp

def find_xy_gates(n):
    str_n = str(n).zfill(2)
    inds_x = find_ind_g('x' + str_n)
    inds_y = find_ind_g('y' + str_n)
    return list(set(inds_x).union(set(inds_y)))

for ind_g in find_xy_gates(0):
    print_queue.append(ind_g)

n = 0
while print_queue:
    ind_g = print_queue.popleft()
    print(gates[ind_g])

    if len(print_queue) == 0:
        n += 1
        for ind_g in find_xy_gates(n):
            print_queue.append(ind_g)
        if len(print_queue) > 0:
            assert len(print_queue) == 2


('x00', 'y00', 'rfg', 'AND')
('x00', 'y00', 'z00', 'XOR')
('y01', 'x01', 'fps', 'XOR')
('x01', 'y01', 'hnq', 'AND')
('x02', 'y02', 'hvw', 'AND')
('x02', 'y02', 'nfs', 'XOR')
('x03', 'y03', 'dcf', 'AND')
('x03', 'y03', 'qtf', 'XOR')
('x04', 'y04', 'ppf', 'AND')
('y04', 'x04', 'rqm', 'XOR')
('x05', 'y05', 'frt', 'AND')
('y05', 'x05', 'wgk', 'XOR')
('y06', 'x06', 'fdv', 'AND')
('x06', 'y06', 'dbp', 'XOR')
('x07', 'y07', 'twn', 'XOR')
('x07', 'y07', 'rnh', 'AND')
('x08', 'y08', 'vvd', 'XOR')
('y08', 'x08', 'ndk', 'AND')
('x09', 'y09', 'ggk', 'AND')
('x09', 'y09', 'pnd', 'XOR')
('x10', 'y10', 'wkd', 'AND')
('y10', 'x10', 'mtg', 'XOR')
('y11', 'x11', 'cdk', 'AND')
('y11', 'x11', 'gcg', 'XOR')
('x12', 'y12', 'gnr', 'AND')
('y12', 'x12', 'mtq', 'XOR')
('y13', 'x13', 'fcm', 'XOR')
('y13', 'x13', 'nhj', 'AND')
('y14', 'x14', 'prf', 'AND')
('y14', 'x14', 'ths', 'XOR')
('x15', 'y15', 'fqf', 'XOR')
('x15', 'y15', 'z15', 'AND')
('x16', 'y16', 'wfs', 'AND')
('x16', 'y16', 'ftv', 'XOR')
('y17', 'x17',

In [46]:
# w = 'chk'

w = 'vpn'

# w = 'z15'
# w = 'qbw'

print('IN')
for ind_g in dict_in[w]:
    print(gates[ind_g])


print('OUT')
for ind_g in dict_out[w]:
    print(gates[ind_g])

IN
('vpn', 'mtg', 'pbg', 'AND')
('vpn', 'mtg', 'z10', 'XOR')
OUT
('jpj', 'ggk', 'vpn', 'OR')
